# Growth rate analysis with Bioscreen or any multi-well plate

This notebook contain workflow to prepare and analyze the results of microbial growth in Bioscreen plate or any multi-well plate.

------

Lidia Lipińska-Zubrycka, Institute of Genetics and Biotechnology

In [83]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime as DT
from datetime import timedelta
import parsedatetime
from pytimeparse import parse
     

In [ ]:
# Define the function to preprocess data

def join_rawdata_and_metadata(rawdata_path, metadata_path, output_path = 'Bioscreen_output_table.csv'):
    '''
    Funkcja przekształca tabelę wyników Bioscreen - dodaje poprawne nazwy kolumn na podstawie 
    pliku csv z metadanymi oraz przekształca format kolumny 'Time'.
    Input:
        rawdata_path    str, sciezka do pliku z danymi
        metadata_path   str, sciezka do pliku z metadanymi (nazwy prob na plytce Bioscreen)
    Output:
        output_path     str, sciezka do pliku po przekształceniu 
    
    '''
    

    metadata = pd.read_csv(metadata_path, sep = ';')
    raw_results = pd.read_csv(rawdata_path, sep=',')
    column_names = pd.concat([metadata[col_name] for col_name in metadata.columns],ignore_index=True)

    results = raw_results.drop(['Time'], axis=1)
    if results.shape[1] == column_names.shape[0]:
        results.columns = column_names
    else: 
        results = results.iloc[: , :column_names.shape[0]]
        print('Tabela z metadanymi zawiera {} prób, a tabela z wynikami {} prób.\n Tabela z wynikami zawiera {} prób. \n Aby otrzymać pełną tabelę wyników, uzupełnij tabelę z metadanymi')

    time = raw_results['Time']
    for i in range(len(time)):
        time[i] = parse(time[i], '%H:%M:%S')
    time = time/3600.0
    results2= results.set_index(time)
    results2.to_csv(output_path)
    #if results.shape[1] >100:

In [85]:
## Sprawdzenie skryptu

join_rawdata_and_metadata(rawdata_path= 'danemaciek_kwiecien2022/22.04.06.csv', 
                          metadata_path = 'danemaciek_kwiecien2022/metadane (1).csv', 
                          output_path = 'danemaciek_kwiecien2022/Bioscreen_output_table.csv')

<ipython-input-83-a7d349962b7e>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time[i] = parse(time[i], '%H:%M:%S')


In [113]:
## Przekształcenie tabeli do podłużnego formatu, np. do przygotowania wykresów

res2 = results.reset_index()
results_longer_df = pd.melt(res2, value_vars=results.columns, id_vars = 'Time',
                           value_name='OD', var_name='sample')#, value_vars=None, var_name=None, value_name='value'
#results_longer_df['sample2'] = results_longer_df['sample'].apply(lambda x: x.split(r'\.\d$')[0])
#results_longer_df['sample_ogolnie'] = results_longer_df['sample'].apply(lambda x: x.split('_')[-1])
